<a href="https://colab.research.google.com/github/cemredogan-ceng/BUSI/blob/main/busi_googlenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Install necessary libraries
!pip install segmentation-models-pytorch
!pip install torch torchvision
!pip install scikit-learn

# Import libraries
import os
from google.colab import drive
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models import googlenet, GoogLeNet_Weights
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from PIL import Image, ImageEnhance
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

# Mount Google Drive
drive.mount('/content/drive')

# Dataset directory
base_dir = '/content/drive/My Drive/Dataset_BUSI_with_GT'

# Custom Dataset Class
class BreastUltrasoundDataset(Dataset):
    def __init__(self, root_dir, classes=['benign', 'malignant'], transform_image=None, transform_mask=None):
        self.root_dir = root_dir
        self.classes = classes
        self.transform_image = transform_image
        self.transform_mask = transform_mask
        self.image_paths = []
        self.mask_paths = []

        # Load image and mask paths
        for cls in self.classes:
            image_dir = os.path.join(root_dir, cls)
            for filename in os.listdir(image_dir):
                if filename.endswith(".png") and "_mask" not in filename:
                    image_path = os.path.join(image_dir, filename)
                    mask_path = os.path.join(image_dir, filename.replace(".png", "_mask.png"))
                    self.image_paths.append(image_path)
                    self.mask_paths.append(mask_path)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        mask = Image.open(self.mask_paths[idx]).convert("L")  # Grayscale mask

        # Apply data augmentations
        if np.random.rand() > 0.5:
            image = ImageEnhance.Contrast(image).enhance(1.5)  # Increase contrast
        if np.random.rand() > 0.5:
            image = image.transpose(Image.FLIP_LEFT_RIGHT)  # Horizontal flip
            mask = mask.transpose(Image.FLIP_LEFT_RIGHT)
        if np.random.rand() > 0.5:
            image = image.transpose(Image.FLIP_TOP_BOTTOM)  # Vertical flip
            mask = mask.transpose(Image.FLIP_TOP_BOTTOM)

        # Apply transformations
        if self.transform_image:
            image = self.transform_image(image)
        if self.transform_mask:
            mask = self.transform_mask(mask)

        return image, mask

# Image and mask transformations
transform_image = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

transform_mask = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Create dataset and dataloader
dataset = BreastUltrasoundDataset(
    root_dir=base_dir,
    classes=['benign', 'malignant'],
    transform_image=transform_image,
    transform_mask=transform_mask
)

train_loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

# Custom GoogleNet-based segmentation model
class GoogleNetSegmentationModel(nn.Module):
    def __init__(self, weights=GoogLeNet_Weights.IMAGENET1K_V1, num_classes=1):
        super(GoogleNetSegmentationModel, self).__init__()
        # Load pretrained GoogleNet with aux_logits=True
        self.encoder = googlenet(weights=weights, aux_logits=True)

        # Manually remove auxiliary classifiers
        self.encoder.aux1 = None
        self.encoder.aux2 = None

        # Remove avgpool and classification head
        self.encoder.avgpool = nn.Identity()
        self.encoder.fc = nn.Identity()

        # Decoder head
        self.decoder = nn.Sequential(
            nn.Conv2d(1024, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True),
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=4, mode="bilinear", align_corners=True),  # Upsample to match input size
            nn.Conv2d(64, num_classes, kernel_size=1),  # Final output
        )

    def forward(self, x):
        x = F.interpolate(x, size=(224, 224), mode="bilinear", align_corners=False)  # Resize for GoogleNet
        features = self.encoder(x)

        # Extract the main output from GoogLeNetOutputs
        if isinstance(features, tuple):  # GoogLeNetOutputs returns (main_output, aux1, aux2)
            features = features[0]  # Use only the main output

        # Reshape features to spatial dimensions
        batch_size = x.size(0)
        features = features.view(batch_size, 1024, 7, 7)  # Reshape to [batch_size, channels, height, width]

        decoded = self.decoder(features)

        # Resize the decoder output to match the target mask size (256x256)
        decoded = F.interpolate(decoded, size=(256, 256), mode="bilinear", align_corners=True)

        return decoded

# Model initialization
model = GoogleNetSegmentationModel(weights=GoogLeNet_Weights.DEFAULT, num_classes=1).cuda()

# Loss function and optimizer
loss_function = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training Loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, masks in train_loader:
        images, masks = images.cuda(), masks.cuda()
        optimizer.zero_grad()
        predictions = model(images)
        loss = loss_function(predictions, masks)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(train_loader)}")

# Save the trained model
torch.save(model.state_dict(), '/content/drive/My Drive/Dataset_BUSI_with_GT/googlenet_segmentation_model.pth')

# Evaluation Function
def evaluate_model(dataset, model, num_samples=100):
    model.eval()
    all_preds = []
    all_masks = []

    with torch.no_grad():
        for i in range(min(num_samples, len(dataset))):
            image, mask = dataset[i]
            image = image.unsqueeze(0).cuda()  # Add batch dimension
            mask = mask.squeeze().cpu().numpy()  # Ground truth mask in NumPy format

            # Get model prediction
            prediction = model(image)
            prediction = torch.sigmoid(prediction).cpu().squeeze().numpy()  # Apply sigmoid activation

            # Binarize the prediction with a threshold of 0.5
            prediction_binary = (prediction > 0.5).astype(np.uint8)

            # Flatten the arrays for metric computation
            all_preds.append(prediction_binary.flatten())
            all_masks.append(mask.flatten())

    # Concatenate all predictions and masks
    y_pred = np.concatenate(all_preds)
    y_true = np.concatenate(all_masks).astype(np.uint8)

    # Compute metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=1)
    recall = recall_score(y_true, y_pred, zero_division=1)
    f1 = f1_score(y_true, y_pred, zero_division=1)

    # Dice score
    intersection = np.logical_and(y_true, y_pred).sum()
    dice_score = (2.0 * intersection) / (y_true.sum() + y_pred.sum() + 1e-6)

    # IoU (Jaccard Index)
    union = np.logical_or(y_true, y_pred).sum()
    iou = intersection / (union + 1e-6)

    # Print metrics
    print("Evaluation Metrics:")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1 Score: {f1:.4f}")
    print(f"  Dice Score: {dice_score:.4f}")
    print(f"  IoU (Jaccard Index): {iou:.4f}")

# Evaluate the entire dataset
evaluate_model(dataset, model, num_samples=len(dataset))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/20, Loss: 0.3551382269801163
Epoch 2/20, Loss: 0.20532510157038525
Epoch 3/20, Loss: 0.17702035642251734
Epoch 4/20, Loss: 0.1565091980666649
Epoch 5/20, Loss: 0.146095191378419
Epoch 6/20, Loss: 0.12459681146755451
Epoch 7/20, Loss: 0.11158780771784665
Epoch 8/20, Loss: 0.1034248914660477
Epoch 9/20, Loss: 0.09738779758534781
Epoch 10/20, Loss: 0.08952094914346206
Epoch 11/20, Loss: 0.08720068424576666
Epoch 12/20, Loss: 0.08249798862308991
Epoch 13/20, Loss: 0.08104797952422281
Epoch 14/20, Loss: 0.0769011114792126
Epoch 15/20, Loss: 0.07395741770543703
Epoch 16/20, Loss: 0.07001043356409888
Epoch 17/20, Loss: 0.06636004322549192
Epoch 18/20, Loss: 0.06470111884721895
Epoch 19/20, Loss: 0.06106744652114263
Epoch 20/20, Loss: 0.06422481122540265
Evaluation Metrics:
  Accuracy: 0.9784
  Precision: 0.8855
  Recall: 0.8707
  F1 Score: 0.8780
  Dice Scor